```
水文水質データベース　Water Information System
```
　　　http://www1.river.go.jp/
    
```

HP > 関連資料 > 水文観測業務規程細則
を参考に、日本全国の国土交通省の公開する観測所データを集計。

(2021年４月１３日作成)
```

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pickle

In [ ]:
def access_MLIT_website(_id):
    url=f"http://www1.river.go.jp/cgi-bin/SiteInfo.exe?ID={_id}"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    info_list=soup.find_all("tr")
    return info_list

def exsistance_checker(info_list):
    if len(info_list)>0:
        return True
    else:
        False

def find(info_list):
    for info_name in info_list:
        info_name_str=str(info_name)
        print(re.findall('<td width="150">(.*)</td>', info_name_str))

In [ ]:
station_class_code_list=[str(i) for i in range(1,6)]

basin_code_raw=pd.read_csv("basin_code_list.csv",header=None).values[0,0]
basin_code_list=basin_code_raw.split(" ")
basin_code_list.sort()

jurisdiction_code_list=["12","14","19"]

station_number_code_list=["{0:03d}".format(i) for i in range(0,1000,1)]

In [ ]:
jimusyo_code_raw_list=pd.read_csv("事務所.csv",header=None).values[:,0]

jimusyo_code_list=[]
for jimusyo_code_raw in jimusyo_code_raw_list:
    jimusyo_code_list.append(jimusyo_code_raw.split(" "))
jimusyo_code_list_=[]
for i in jimusyo_code_list:
    for j in i:
        jimusyo_code_list_.append(j)

jimusyo_code_list_chosen=[]
for i in jimusyo_code_list_:
    try:
        int(i)
        jimusyo_code_list_chosen.append(i)
    except:
        pass

jimusyo_code_list_chosen.sort()


In [ ]:
local_bureau_code_list=["{0:02d}".format(i) for i in range(81,91)]+["98","99"]

In [ ]:
area_list=["北海道","東北","関東","北陸","中部","近畿","中国","四国","九州"]

In [ ]:
area_base_code_dict={i:{"水系":[],"地方整備局":[],"事務所":[],} for i in area_list}

In [ ]:
area_base_code_dict

In [ ]:
for i in basin_code_list:
    if i[:2]=="01":
        area_base_code_dict["北海道"]["水系"].append(i)
    elif i[:2]=="02":
        area_base_code_dict["東北"]["水系"].append(i)
    elif i[:2]=="03":
        area_base_code_dict["関東"]["水系"].append(i)
    elif i[:2]=="04":
        area_base_code_dict["北陸"]["水系"].append(i)
    elif i[:2]=="05":
        area_base_code_dict["中部"]["水系"].append(i)
    elif i[:2]=="06":
        area_base_code_dict["近畿"]["水系"].append(i)
    elif i[:2]=="07":
        area_base_code_dict["中国"]["水系"].append(i)
    elif i[:2]=="08":
        area_base_code_dict["四国"]["水系"].append(i)
    elif i[:2]=="09":
        area_base_code_dict["九州"]["水系"].append(i)


In [ ]:
for i in local_bureau_code_list:
    if i=="81":
        area_base_code_dict["北海道"]["地方整備局"].append(i)
    elif i=="82":
        area_base_code_dict["東北"]["地方整備局"].append(i)
    elif i=="83":
        area_base_code_dict["関東"]["地方整備局"].append(i)
    elif i=="84":
        area_base_code_dict["北陸"]["地方整備局"].append(i)
    elif i=="85":
        area_base_code_dict["中部"]["地方整備局"].append(i)
    elif i=="86":
        area_base_code_dict["近畿"]["地方整備局"].append(i)
    elif i=="87":
        area_base_code_dict["中国"]["地方整備局"].append(i)
    elif i=="88":
        area_base_code_dict["四国"]["地方整備局"].append(i)
    elif i=="89":
        area_base_code_dict["九州"]["地方整備局"].append(i)    

In [ ]:
for i in jimusyo_code_list_chosen:
    if i[0]=="1":
        area_base_code_dict["北海道"]["事務所"].append(i)
    elif i[0]=="2":
        area_base_code_dict["東北"]["事務所"].append(i)
    elif i[0]=="3":
        area_base_code_dict["関東"]["事務所"].append(i)
    elif i[0]=="4":
        area_base_code_dict["北陸"]["事務所"].append(i)
    elif i[0]=="5":
        area_base_code_dict["中部"]["事務所"].append(i)
    elif i[0]=="6":
        area_base_code_dict["近畿"]["事務所"].append(i)
    elif i[0]=="7":
        area_base_code_dict["中国"]["事務所"].append(i)
    elif i[0]=="8":
        area_base_code_dict["四国"]["事務所"].append(i)
    elif i[0]=="9":
        area_base_code_dict["九州"]["事務所"].append(i)


In [ ]:
df=pd.DataFrame(columns=['観測所名','観測項目','観測所記号','水系名','河川名','所在地','緯度経度'])
COUNTER=-1
"""
station_class_code
1: 雨量
3: 水位、流量
"""
for station_class_code in [1,3]:
    for jurisdiction_code in jurisdiction_code_list:
        for area in area_list:
            basin_code_list=area_base_code_dict[area]["水系"]
            local_bureau_code_list=area_base_code_dict[area]["地方整備局"]
            jimusyo_code_list=area_base_code_dict[area]["事務所"]
            for basin_code in basin_code_list:
                for local_bureau_code in local_bureau_code_list:
                    for jimusyo_code in jimusyo_code_list:
                        for station_number_code in station_number_code_list:
                            _id=f"{station_class_code}{basin_code}{jurisdiction_code}{local_bureau_code}{jimusyo_code}{station_number_code}"
                            info_list=access_MLIT_website(_id)
                            if exsistance_checker(info_list):
                                COUNTER+=1
                                for info in info_list:
                                    info_str=str(info)
                                    colname=re.findall('<td width="150">(.*)</td>', info_str)
                                    if exsistance_checker(colname):
                                        if colname==["緯度経度"]:
                                            if station_class_code==3:
                                                value=re.findall('<td align="LEFT" colspan="2">(.*)</td>', info_str)
                                            else:
                                                value=re.findall('<td align="LEFT">(.*)</td>', info_str)
                                        else:
                                            value=re.findall('<td colspan="2">(.*)</td>', info_str)
                                        if value==[]:
                                            value=""
                                        try:
                                            df.loc[COUNTER,colname]=value
                                        except KeyError:
                                            pass
                                        print(colname,value)    
#             pickle.dump(df,open(f"df_rain_{area}.pkl","wb"))
#             pickle.dump(df,open(f"df_level_flow_{area}.pkl","wb"))
            pickle.dump(df,open(f"df_rain_level_flow_2.pkl","wb"))